In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from tqdm import tqdm
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
from bs4 import BeautifulSoup
import lxml
import re
from uuid import uuid4
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.exc import IntegrityError

In [7]:
class Crawler:
        
    def get_html(self):
        pass
    
    def get_card_info(self):
        pass

In [8]:
class CrawlerCian(Crawler):
    
    def get_html(self, page):
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")

        browser = webdriver.Chrome(options=chrome_options)
        cian_url = f"https://chelyabinsk.cian.ru/cat.php?deal_type=sale&engine_version=2&offer_type=flat&p={page}&region=5048"
        browser.get(cian_url)
        wait = WebDriverWait(browser, 10)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        return soup

    
    def get_card_info(self, number):
        chrome_options = Options()
        chrome_options.add_argument("--disable-extensions")
        chrome_options.add_argument("--disable-popup-blocking")
        
        browser = webdriver.Chrome(options=chrome_options)
        action = ActionChains(browser)
        count = 0
        
        linker = linkers[number]['link']
        browser.get(linker)
        wait = WebDriverWait(browser, 10)
        soup_card = BeautifulSoup(browser.page_source, 'html.parser')
        return soup_card

In [9]:
class Parser:
    def __init__(self, html):
        self.html = html
    
    def get_cards(self, html):
        pass

In [10]:
class ParserCian(Parser):
    
    def get_links_cards(self):
        lst = []
        lst_of_links = self.html.find_all('div', '_93444fe79c--container--kZeLu _93444fe79c--link--DqDOy')
        for link in lst_of_links:
            query = self.get_linkers(link)
            lst.append(query)
        return lst
    
    def get_cards(self):
        lst_of_cards = self.html.find_all('div', 'a10a3f92e9--page--OYngf')
        lst = []
        for card in lst_of_cards:
            query = self.get_info(card)
            lst.append(query)
        return lst
    
    def __get_links(self, card):
        links = card.find_all('a', class_ = '_93444fe79c--link--eoxce', href=True)
        origin_links = [link['href'] for link in links]
        return origin_links[0]
    
    def __get_address(self, card):
        origin_addresses = card.find('div', class_ = 'a10a3f92e9--address-line--GRDTb').text
        end_address = origin_addresses.replace('На карте', '')
        return end_address
    
    def __get_views(self, card):
        origin_views = card.find('button', class_ = 'a10a3f92e9--button--lyQVM').text
        return origin_views
    
    def __get_сosts(self, card):
        costs = card.find('div', class_ = 'a10a3f92e9--amount--ON6i1').text
        new_costs = costs.replace('₽', '')
        origin_costs = new_costs.replace('\xa0', '')
        return origin_costs
    
    def __get_square_costs(self, card):
        square_costs = card.find('div', class_ = 'a10a3f92e9--price_per_meter--yfcbi a10a3f92e9--price_per_meter--residential--DUaaw')
        if square_costs:
            new_square_costs = square_costs.text.replace('₽/м²', '')
            origin_square_costs = new_square_costs.replace('\xa0', '')
            end_square_costs = origin_square_costs.replace('Цена за метр', '')
        else:
            square_costs = card.find('div', class_ = 'a10a3f92e9--item--iWTsg')
            new_square_costs = square_costs.text.replace('₽/м²', '')
            origin_square_costs = new_square_costs.replace('\xa0', '')
            end_square_costs = origin_square_costs.replace('Цена за метр', '')
        return end_square_costs
    
    def __get_full_square(self, card):
        full_square = card.find_all('div', class_ = 'a10a3f92e9--text--eplgM')
        origin_full_costs = full_square[0].text.replace('\xa0', '')
        end_full_square = origin_full_costs.replace('Общая площадь', '')
        return end_full_square
    
    def __get_flat_info(self, card):
        another_info = card.find('div', class_ = 'a10a3f92e9--group--K5ZqN')
        if another_info:
            origin_another_info = another_info.text.replace('\xa0', '')
            origin_info = origin_another_info.replace('О квартире', '')
            s = re.sub(r'(?<=\w)([А-ЯЁ])', r' \1', origin_info)
            s = s.replace('Тип жилья', 'Тип жилья:')
            s = s.replace('Общая площадь', 'Общая площадь:')
            s = s.replace('Высота потолков', 'Высота потолков:')
            s = s.replace('Отделка', 'Отделка:')
            s = s.replace('Площадь кухни', 'Площадь кухни:')
            s = s.replace('Жилая площадь', 'Жилая площадь:')
        else:
            another_info = card.find('div', class_ = 'a10a3f92e9--list--jHl8z')
            origin_info = origin_another_info.replace('О квартире', '')
            s = re.sub(r'(?<=\w)([А-ЯЁ])', r' \1', origin_info)
            s = s.replace('Тип жилья', 'Тип жилья:')
            s = s.replace('Общая площадь', 'Общая площадь:')
            s = s.replace('Высота потолков', 'Высота потолков:')
            s = s.replace('Отделка', 'Отделка:')
            s = s.replace('Планировка', 'Планировка:')
            s = s.replace('Площадь комнат', 'Площадь комнат:')
            s = s.replace('Санузел', 'Санузел:')
            s = s.replace('Балкон/лоджия', 'Балкон/лоджия:')
            s = s.replace('Вид из окон', 'Вид из окон:')
            s = s.replace('Ремонт', 'Ремонт:')
        return s

    def __get_house_info(self, card):
        another_info = card.find('div', class_ = 'a10a3f92e9--group--K5ZqN a10a3f92e9--right--_9uBM')
        if another_info:
            origin_another_info = another_info.text.replace('\xa0', '')
            origin_info = origin_another_info.replace('О доме', '')
            s = re.sub(r'(?<=\w)([А-ЯЁ])', r' \1', origin_info)
            s = s.replace('Количество лифтов', 'Количество лифтов:')
            s = s.replace('Тип дома', 'Тип дома:')
            s = s.replace('Подписаться на дом ', '')
        else:
            s = None
        return s
    
    def get_linkers(self, html_of_cards):
        links_dict = {'link':self.__get_links(html_of_cards)}
        return links_dict
            
    def get_info(self, html_of_cards):

        flat_dict = {'address':self.__get_address(html_of_cards),
                'price': self.__get_сosts(html_of_cards),
                'price_per_sqm': self.__get_square_costs(html_of_cards),
                'views':self.__get_views(html_of_cards),
                'area': self.__get_full_square(html_of_cards),
                'flat_info': self.__get_flat_info(html_of_cards),
                'house_info': self.__get_house_info(html_of_cards),
        }
        return flat_dict

In [11]:
Base = declarative_base()

class Flat(Base):
    __tablename__ = "flats"

    id = Column(Integer, primary_key=True)
    address = Column(String)
    price = Column(String)
    price_per_sqm = Column(String)
    views = Column(String)
    area = Column(String)
    flat_info = Column(String)
    house_info = Column(String)
    link = Column(String, unique=True)
    latitudes = Column(Float)
    longitudes = Column(Float)

In [12]:
class DBHelper:
    def __init__(self, db_name):
        self.engine = create_engine(f"sqlite:///{db_name}")
        self.session = sessionmaker(bind=self.engine)()

    def create_table(self):
        Base.metadata.create_all(self.engine)

    def insert(self, data):
        for flat_data in data:
            flat = Flat(**flat_data)
            self.session.add(flat)
            try:
                self.session.commit()
            except IntegrityError as e:
                self.session.rollback()
                continue

    def print_info(self):
        flats = self.session.query(Flat).all()
        for flat in flats:
            print("ID:", flat.id)
            print("address:", flat.address)
            print("Стоимость:", flat.price)
            print("Стоимость кв/м2:", flat.price_per_sqm)
            print("Количество просмотров:", flat.views)
            print("Общая площадь:", flat.area)
            print("Информация о квартире:", flat.flat_info)
            print("Информация о доме:", flat.house_info)
            print("Ссылка:", flat.link)
            print("Широта:", flat.latitudes)
            print("Долгота:", flat.longitudes)
            print()

    def update(self, flat_id, updates):
        flat = self.session.query(Flat).filter_by(id=flat_id).first()
        if flat:
            for key, value in updates.items():
                setattr(flat, key, value)
            self.session.commit()
        else:
            print("Запись не найдена")

    def delete(self, flat_id):
        flat = self.session.query(Flat).filter_by(id=flat_id).first()
        if flat:
            self.session.delete(flat)
            self.session.commit()
        else:
            print("Запись не найдена")

In [13]:
pars_cian = CrawlerCian()

In [14]:
linkers = []
for page in range(0, 53):
    cian_html = pars_cian.get_html(page)
    cian = ParserCian(cian_html)
    linkers += cian.get_links_cards()

In [15]:
print(len(linkers))
print(linkers)

1483
[{'link': 'https://chelyabinsk.cian.ru/sale/flat/288354055/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/283451846/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/280787695/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/288463333/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/288368886/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/283951738/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/272977473/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/284785887/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/272977626/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/284332685/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/287925583/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/281873664/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/288368932/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/286805209/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/282483940/'}, {'link': 'https://chelyabinsk.cian.ru/sale/flat/284716894/'}, {'

In [16]:
cian_cards = []
for number in range(400, len(linkers)):
    cian_cards_info = pars_cian.get_card_info(number)
    cian_card = ParserCian(cian_cards_info)
    cian_cards += cian_card.get_cards()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=114.0.5735.111)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00F0A813+48355]
	(No symbol) [0x00E9C4B1]
	(No symbol) [0x00DA5358]
	(No symbol) [0x00D8D293]
	(No symbol) [0x00DEE37B]
	(No symbol) [0x00DFC473]
	(No symbol) [0x00DEA536]
	(No symbol) [0x00DC82DC]
	(No symbol) [0x00DC93DD]
	GetHandleVerifier [0x0116AABD+2539405]
	GetHandleVerifier [0x011AA78F+2800735]
	GetHandleVerifier [0x011A456C+2775612]
	GetHandleVerifier [0x00F951E0+616112]
	(No symbol) [0x00EA5F8C]
	(No symbol) [0x00EA2328]
	(No symbol) [0x00EA240B]
	(No symbol) [0x00E94FF7]
	BaseThreadInitThunk [0x75C47D59+25]
	RtlInitializeExceptionChain [0x770AB74B+107]
	RtlClearBits [0x770AB6CF+191]


In [17]:
addresses = []
for i in range(len(cian_cards)):
    addresses.append(cian_cards[i]['address'])

In [18]:
from geopy.geocoders import Yandex
geolocator = Yandex(api_key='6e579ee3-462f-44ad-a422-548c76c36119')

latitudes = []
longitudes = []
for address in addresses:
    location = geolocator.geocode(address, timeout=10)
    if location is not None:
        latitudes.append(location.latitude)
        longitudes.append(location.longitude)

In [19]:
for i in range(len(cian_cards)):
    cian_cards[i].update(linkers[i])
    cian_cards[i]["latitudes"] = latitudes[i]
    cian_cards[i]["longitudes"] = longitudes[i]

In [20]:
cian_cards

[{'address': 'Челябинская область, Челябинск, р-н Центральный, ул. Труда, 157А',
  'price': '22512600',
  'price_per_sqm': '198 000 ',
  'views': '423 просмотра, 2 за сегодня',
  'area': '113,7м²',
  'flat_info': 'Тип жилья: Новостройка Апартаменты Общая площадь:113,7 Высота потолков:2,85м Отделка: Черновая',
  'house_info': 'Количество лифтов:4 грузовых Тип дома: Монолитный',
  'link': 'https://chelyabinsk.cian.ru/sale/flat/288354055/',
  'latitudes': 55.166588,
  'longitudes': 61.390938},
 {'address': 'Челябинская область, Челябинск, р-н Курчатовский, мкр. 57-й, ул. Маршала Чуйкова, 21',
  'price': '2440000',
  'price_per_sqm': '104 274 ',
  'views': '19 просмотров, 0 за сегодня',
  'area': '23,4м²',
  'flat_info': 'Тип жилья: Новостройка Общая площадь:23,4 Санузел1 совмещенный Балкон/лоджия1 лоджия',
  'house_info': 'Тип дома: Панельный',
  'link': 'https://chelyabinsk.cian.ru/sale/flat/283451846/',
  'latitudes': 55.214548,
  'longitudes': 61.293067},
 {'address': 'Челябинская обла

In [21]:
db = DBHelper('cian_parser.db')

In [22]:
db.create_table()

In [23]:
db.insert(cian_cards)

In [24]:
db.print_info()

ID: 1
address: Челябинская область, Сосновский район, Западный поселок, Привилегия микрорайон
Стоимость: 8400000
Стоимость кв/м2: 104 856 
Количество просмотров: 337 просмотров, 14 за сегодня
Общая площадь: 80,11м²
Информация о квартире: Тип жилья: Новостройка Общая площадь:80,11 Высота потолков:2,7м Отделка: Черновая
Информация о доме: Количество лифтов:1 грузовой Тип дома: Кирпичный
Ссылка: https://chelyabinsk.cian.ru/sale/flat/280787695/
Широта: 55.108256
Долгота: 61.249732

ID: 2
address: Челябинская область, Сосновский район, Терема поселок, ул. Ломоносова, 22
Стоимость: 2750000
Стоимость кв/м2: 107 422 
Количество просмотров: 206 просмотров, 3 за сегодня
Общая площадь: 25,6м²
Информация о квартире: Тип жилья: Вторичка Общая площадь:25,6 Санузел1 совмещенный Балкон/лоджия1 лоджия Ремонт Без ремонта
Информация о доме: Год постройки2023 Строительная серия Индивидуальный проект Количество лифтов:2 пассажирских Тип перекрытий Нет информации Отопление Нет информации Аварийность Нет
Ссы

Стоимость кв/м2: 76 173 
Количество просмотров: 133 просмотра, 4 за сегодня
Общая площадь: 66,69м²
Информация о квартире: Тип жилья: Новостройка Общая площадь:66,69 Отделка: Предчистовая
Информация о доме: Тип дома: Панельный
Ссылка: https://chelyabinsk.cian.ru/sale/flat/277735433/
Широта: 55.213515
Долгота: 61.295986

ID: 62
address: Челябинская область, Сосновский район, Западный поселок, Вишневая Горка микрорайон, ул. Имени Генерала Костицына, 1
Стоимость: 1850000
Стоимость кв/м2: 75 203 
Количество просмотров: 91 просмотр, 6 за сегодня
Общая площадь: 24,6м²
Информация о квартире: Тип жилья: Вторичка Общая площадь:24,6 Жилая площадь:2,5 Площадь кухни:14 Высота потолков:2,5м Санузел1 совмещенный Балкон/лоджия1 лоджия Ремонт Косметический
Информация о доме: Год постройки2020 Количество лифтов:1 грузовой Тип дома: Панельный Тип перекрытий Нет информации Парковка Наземная Подъезды3 Отопление Центральное Аварийность Нет
Ссылка: https://chelyabinsk.cian.ru/sale/flat/287903483/
Широта: 55.

Информация о доме: Количество лифтов:1 грузовой Тип дома: Панельный
Ссылка: https://chelyabinsk.cian.ru/sale/flat/287534267/
Широта: 55.127224
Долгота: 61.260548

ID: 106
address: Челябинская область, Челябинск, р-н Курчатовский, мкр. А, ул. Красного Урала, 13
Стоимость: 3700000
Стоимость кв/м2: 58 452 
Количество просмотров: 605 просмотров, 6 за сегодня
Общая площадь: 63,3м²
Информация о квартире: Тип жилья: Вторичка Общая площадь:63,3 Площадь кухни:6 Санузел1 совмещенный, 1 раздельный Балкон/лоджия1 балкон Вид из окон На улицу и двор Ремонт Косметический
Информация о доме: Год постройки1973 Строительная серия1-447 Мусоропровод Нет Тип дома: Блочный Тип перекрытий Железобетонные Подъезды8 Отопление Центральное Аварийность Нет Газоснабжение Центральное
Ссылка: https://chelyabinsk.cian.ru/sale/flat/284652251/
Широта: 55.196218
Долгота: 61.355042

ID: 107
address: Челябинская область, Сосновский район, Пригородный поселок, Парковый Премиум жилой комплекс
Стоимость: 5096560
Стоимость кв/м

address: Челябинская область, Сосновский район, Рощино поселок
Стоимость: 2064800
Стоимость кв/м2: 58 000 
Количество просмотров: 24 просмотра, 1 за сегодня
Общая площадь: 35,6м²
Информация о квартире: Тип жилья: Новостройка Общая площадь:35,6 Высота потолков:2,64м Отделка: Чистовая
Информация о доме: Количество лифтов:1 грузовой Тип дома: Панельный
Ссылка: https://chelyabinsk.cian.ru/sale/flat/286910427/
Широта: 55.31056
Долгота: 61.263396

ID: 151
address: Челябинская область, Челябинск, р-н Курчатовский, мкр. 4-й, ул. Молодогвардейцев, 23
Стоимость: 3480000
Стоимость кв/м2: 56 311 
Количество просмотров: 84 просмотра, 9 за сегодня
Общая площадь: 61,8м²
Информация о квартире: Тип жилья: Вторичка Общая площадь:61,8 Площадь кухни:6 Санузел1 раздельный Балкон/лоджия1 балкон Ремонт Косметический
Информация о доме: Год постройки1969 Строительная серия Т К Б У Мусоропровод Нет Тип дома: Панельный Тип перекрытий Железобетонные Подъезды8 Отопление Центральное Аварийность Нет Газоснабжение Це

Информация о квартире: Тип жилья: Новостройка Общая площадь:55,41 Жилая площадь:23,7 Площадь кухни:5 Высота потолков:2,65м Санузел1 раздельный Балкон/лоджия1 балкон Вид из окон На улицу и двор Отделка: Чистовая
Информация о доме: Количество лифтов:1 грузовой Тип дома: Панельный
Ссылка: https://chelyabinsk.cian.ru/sale/flat/288246646/
Широта: 55.122529
Долгота: 61.247118

ID: 195
address: Челябинская область, Челябинск, р-н Центральный, ул. Сони Кривой, 36
Стоимость: 12800000
Стоимость кв/м2: 150 235 
Количество просмотров: 113 просмотров, 17 за сегодня
Общая площадь: 85,2м²
Информация о квартире: Тип жилья: Вторичка Пентхаус Общая площадь:85,2 Жилая площадь:64 Площадь кухни:8 Высота потолков:2,7м Санузел2 совмещенных Балкон/лоджия2 лоджии Вид из окон Во двор Ремонт Без ремонта
Информация о доме: Год постройки1963 Тип дома: Блочный Тип перекрытий Железобетонные Парковка Наземная Подъезды2 Отопление Центральное Аварийность Нет Газоснабжение Центральное
Ссылка: https://chelyabinsk.cian.ru

Долгота: 61.249732

ID: 239
address: Челябинская область, Челябинск, р-н Советский, ул. Заслонова, 1
Стоимость: 1800000
Стоимость кв/м2: 40 632 
Количество просмотров: 322 просмотра, 4 за сегодня
Общая площадь: 44,3м²
Информация о квартире: Тип жилья: Вторичка Общая площадь:44,3 Жилая площадь:29 Площадь кухни:7 Санузел1 раздельный Балкон/лоджия1 балкон Вид из окон На улицу и двор Ремонт Косметический
Информация о доме: Год постройки1959 Строительная серия Индивидуальный проект Тип перекрытий Деревянные Парковка Наземная Подъезды6 Отопление Центральное Аварийность Нет Газоснабжение Центральное
Ссылка: https://chelyabinsk.cian.ru/sale/flat/288659080/
Широта: 55.124254
Долгота: 61.399535

ID: 240
address: Челябинская область, Сосновский район, Западный поселок, Микрорайон Белый хутор
Стоимость: 4530000
Стоимость кв/м2: 71 655 
Количество просмотров: 85 просмотров, 0 за сегодня
Общая площадь: 63,22м²
Информация о квартире: Тип жилья: Новостройка Общая площадь:63,22 Высота потолков:2,6м Отд

Количество просмотров: 103 просмотра, 1 за сегодня
Общая площадь: 80,5м²
Информация о квартире: Тип жилья: Новостройка Общая площадь:80,5 Жилая площадь:59,7 Балкон/лоджия1 лоджия
Информация о доме: None
Ссылка: https://chelyabinsk.cian.ru/sale/flat/288658868/
Широта: 55.214548
Долгота: 61.293067

ID: 284
address: Челябинская область, Челябинск, р-н Калининский, мкр. 29-й, Макеев жилой комплекс
Стоимость: 5800000
Стоимость кв/м2: 95 363 
Количество просмотров: 32 просмотра, 0 за сегодня
Общая площадь: 60,82м²
Информация о квартире: Тип жилья: Новостройка Общая площадь:60,82 Высота потолков:2,8м Отделка: Без отделки
Информация о доме: Количество лифтов:2 грузовых Тип дома: Панельный
Ссылка: https://chelyabinsk.cian.ru/sale/flat/287521236/
Широта: 55.202019
Долгота: 61.44216

ID: 285
address: Челябинская область, Челябинск, р-н Металлургический, мкр. Першино, Аральская ул., 212
Стоимость: 4550000
Стоимость кв/м2: 53 404 
Количество просмотров: 571 просмотр, 9 за сегодня
Общая площадь: 85,

In [3]:
import sys
import sqlite3
import plotly.graph_objs as go
from PyQt5.QtCore import Qt
from PyQt5.QtWidgets import QApplication, QMainWindow, QHBoxLayout, QWidget, QVBoxLayout, QComboBox, QLineEdit, QLabel
from PyQt5.QtWebEngineWidgets import QWebEngineView

In [4]:
import numpy as np
conn = sqlite3.connect('cian_parser.db')
cursor = conn.execute("SELECT * FROM flats")
flats = cursor.fetchall()
    
views = np.array(flats)[:, 4]
numbers = []
for view in views:
    index = view.find('п')
    number = view[:index].replace(' ', '')
    if int(number) <= 100:
        number = int(number) * 5.5
    elif int(number) > 100 and int(number) <= 200:
        number = int(number) * 3.9
    elif int(number) > 200 and int(number) <= 300:
        number = int(number) * 2.3
    elif int(number) > 300 and int(number) <= 400:
        number = int(number) * 1.7
    elif int(number) > 400 and int(number) <= 500:
        number = int(number) * 1.35
    elif int(number) > 500 and int(number) <= 600:
        number = int(number) * 1.1
    numbers.append(int(number))

count_views = np.array(numbers)

In [5]:
class MapWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.map_widget = MapWidget()

        self.central_widget = QWidget()
        self.central_layout = QVBoxLayout(self.central_widget)
        self.central_layout.addWidget(self.map_widget)

        self.price_filter_layout = QHBoxLayout()
        self.min_price_edit = QLineEdit()
        self.min_price_edit.setPlaceholderText('Введите минимальную стоимость')
        self.min_price_edit.returnPressed.connect(self.filter_data_by_min_price)
        self.price_filter_layout.addWidget(QLabel('Минимальная цена:'))
        self.price_filter_layout.addWidget(self.min_price_edit)

        self.max_price_edit = QLineEdit()
        self.max_price_edit.setPlaceholderText('Введите максимальную стоимость')
        self.max_price_edit.returnPressed.connect(self.filter_data_by_max_price)
        self.price_filter_layout.addWidget(QLabel('Максимальная цена:'))
        self.price_filter_layout.addWidget(self.max_price_edit)

        self.area_filter_layout = QHBoxLayout()
        self.min_area_edit = QLineEdit()
        self.min_area_edit.setPlaceholderText('Введите минимальную общую площадь')
        self.min_area_edit.returnPressed.connect(self.filter_data_by_area)
        self.area_filter_layout.addWidget(QLabel('Минимальная м²:'))
        self.area_filter_layout.addWidget(self.min_area_edit)

        self.price_per_sqm_filter_layout = QHBoxLayout()
        self.min_price_per_sqm_edit = QLineEdit()
        self.min_price_per_sqm_edit.setPlaceholderText('Введите минимальную стоимость ₽/м²')
        self.min_price_per_sqm_edit.returnPressed.connect(self.filter_data_by_price_per_sqm)
        self.price_per_sqm_filter_layout.addWidget(QLabel('Минимальная стоимость ₽/м²:'))
        self.price_per_sqm_filter_layout.addWidget(self.min_price_per_sqm_edit)

        self.central_layout.addLayout(self.price_filter_layout)
        self.central_layout.addLayout(self.area_filter_layout)
        self.central_layout.addLayout(self.price_per_sqm_filter_layout)

        self.setCentralWidget(self.central_widget)

        self.setGeometry(100, 100, 800, 600)
        self.setWindowTitle('Карта')

        self.filter_data()

    def filter_data(self):
        self.filter_data_by_min_price()
        self.filter_data_by_area()
        self.filter_data_by_price_per_sqm()
        self.filter_data_by_max_price()

    def filter_data_by_min_price(self):
        min_price_text = self.min_price_edit.text().strip()
        condition = ''
        if min_price_text:
            condition += ' AND price > {}'.format(min_price_text)

        self.map_widget.filter_data(condition)
        
    def filter_data_by_max_price(self):
        max_price_text = self.max_price_edit.text().strip()
        condition = ''
        if max_price_text:
            condition += ' AND price > {}'.format(max_price_text)

        self.map_widget.filter_data(condition)

    def filter_data_by_area(self):
        min_area_text = self.min_area_edit.text().strip()
        condition = ''
        if min_area_text:
            condition += ' AND area > {}'.format(min_area_text)

        self.map_widget.filter_data(condition)

    def filter_data_by_price_per_sqm(self):
        min_price_per_sqm_text = self.min_price_per_sqm_edit.text().strip()
        condition = ''
        if min_price_per_sqm_text:
            condition += ' AND price_per_sqm > {}'.format(min_price_per_sqm_text)

        self.map_widget.filter_data(condition)

class MapWidget(QWidget):
    def __init__(self):
        super().__init__()
        hover_text = []
        
        for flat in flats:
            hover_text.append(f"Адрес: {flat[1]}<br>"
                              f"Стоимость: {flat[2]}<br>"
                              f"Стоимость кв/м2: {flat[3]}<br>"
                              f"Количество просмотров: {flat[4]}<br>"
                              f"Общая площадь: {flat[5]}<br>"
                              f"Информация о квартире: {flat[6][:56]}<br>"
                              f"Ссылка: <a href='{flat[8]}' style='color: blue; text-decoration: underline;'>{flat[8]}</a>")
            
        self.trace = go.Scattermapbox(
            lat=[], 
            lon=[], 
            mode='markers',
            marker=dict(size=count_views/100, color=count_views, colorscale='Reds', showscale=True),
            hoverinfo='text',
            text=hover_text
        )

        layout = go.Layout(
            mapbox=dict(
                accesstoken='pk.eyJ1IjoicHJhemRuaSIsImEiOiJjbGl3c29odGgwMTRsM2RwNmFqMW56Mmo4In0.2BNTBUrwAi1_wQjVh_LIcA', 
                zoom=10, 
                center=dict(lat=55.1540200, lon=61.4291500)
            ),
        )
        
        self.fig = go.Figure(data=[self.trace], layout=layout)

        self.map_view = QWebEngineView(self)
        self.map_view.setHtml(self.fig.to_html(include_plotlyjs='cdn'))

        self.map_layout = QHBoxLayout()
        self.map_layout.addWidget(self.map_view)

        self.setLayout(self.map_layout)
        self.update_data()

    def update_data(self):
        conn = sqlite3.connect('cian_parser.db')
        cur = conn.cursor()
        cur.execute('SELECT latitudes, longitudes, area, price_per_sqm, price FROM flats')
        data = cur.fetchall()
        conn.close()

        self.trace.lat = [item[0] for item in data]
        self.trace.lon = [item[1] for item in data]
        self.fig.update_traces(self.trace)
        self.map_view.setHtml(self.fig.to_html(include_plotlyjs='cdn'))

    def filter_data(self, condition=''):
        conn = sqlite3.connect('cian_parser.db')
        cur = conn.cursor()
        cur.execute('SELECT latitudes, longitudes, area, price_per_sqm, price FROM flats WHERE 1=1{}'.format(condition))
        data = cur.fetchall()
        conn.close()
        self.trace.lat = [item[0] for item in data]
        self.trace.lon = [item[1] for item in data]
        self.fig.update_traces(self.trace)
        self.map_view.setHtml(self.fig.to_html(include_plotlyjs='cdn'))

In [6]:
if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = MapWindow()
    window.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

